<a href="https://colab.research.google.com/github/willhorgalcon/publicrep/blob/main/cats%20and%20dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle competitions download -c dogs-vs-cats


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [17]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iyxohxiyxoy","key":"7f680dd66bbb74508b8f71d7a95373e9"}'}

In [18]:
# !mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
!kaggle competitions download -c dogs-vs-cats


 99% 803M/812M [00:11<00:00, 130MB/s]
100% 812M/812M [00:11<00:00, 71.0MB/s]


In [23]:
!unzip -qq dogs-vs-cats.zip

In [ ]:
!unzip -qq train.zip